In [14]:
import plotly
from plotly.graph_objs import *
import json
import pandas
from pandas.io.json import json_normalize
'''
load data from json file and flatten
'''
datafile="/Users/annejones/chiweb/data/gfs/json/tas-2m-gfs-1.0.json"
print("loading data from" + datafile)
#raw=pandas.read_json(datafile)
#df=json_normalize(raw)
#print(raw['data'].size)
with open(datafile) as f:
    d = json.load(f)
d = json_normalize(d)
df = pandas.DataFrame({'T2m':d['data'].loc[0]})

loading data from/Users/annejones/chiweb/data/gfs/json/tas-2m-gfs-1.0.json


In [15]:
nx=d['header.nx'][0]
ny=d['header.ny'][0]
lo1=d['header.lo1'][0]
lo2=d['header.lo2'][0]
la1=d['header.la1'][0]
la2=d['header.la2'][0]
dx=d['header.dx'][0]
dy=d['header.dy'][0]
ref_time=d['header.refTime'][0]
fcst_time=d['header.forecastTime'][0]
len(df['T2m'])

65160

In [16]:
lo2

359.0

In [17]:
la2

-90.0

In [18]:
import numpy as np
xgrid=np.linspace(start=lo1,stop=lo2,num=nx)
ygrid=np.linspace(start=la1,stop=la2,num=ny)
y=np.repeat(ygrid,nx)
x=np.tile(xgrid,ny)
trace1 = {"x" : x, "y" : y, "z" : df['T2m']-273.15,
  "autocolorscale": False, 
  "colorbar": {"title": "Temperature C"}, 
  "colorscale": [
    [0, "rgb(5, 10, 172)"], [0.35, "rgb(106, 137, 247)"], [0.5, "rgb(190,190,190)"], 
      [0.6, "rgb(220, 170, 132)"], [0.7, "rgb(230, 145, 90)"], [1, "rgb(178, 10, 28)"]], 
  "connectgaps": False, 
  "contours": {
    "coloring": "fill", 
    "end": 45.0, 
    "showlines": True, 
    "size": 2.5, 
    "start": 5.0
  }, 
  "name": "00 UTC", 
  "reversescale": False, 
  "type": "contour", 
  "zauto": False, 
  "zmax": 45.0, 
  "zmin": 5.0
}

In [19]:
data = Data([trace1])
layout = {
  "autosize": False, 
  "height": 800, 
  "legend": {
    "x": 0.861031518625, 
    "y": 0.1
  }, 
  "title": "Temperature at +00h, GFS 1deg Forecast, UTC Start time : 31/01/2018", 
  "width": 1200, 
  "xaxis": {
    "autorange": False, 
    "gridwidth": 1.2, 
    "mirror": True, 
    "range": [lo1, lo2], 
    "showline": True, 
    "ticks": "", 
    "title": "Longitude", 
    "type": "linear", 
    "zeroline": False
  }, 
  "yaxis": {
    "autorange": False, 
    "gridwidth": 1.2, 
    "mirror": True, 
    "range": [la2, la1], 
    "showline": True, 
    "ticks": "", 
    "title": "Latitude", 
    "type": "linear", 
    "zeroline": False
  }
}
fig = Figure(data=data, layout=layout)
plot_url = plotly.offline.plot(fig)

In [20]:
from mpl_toolkits.basemap import Basemap

In [21]:
# https://plot.ly/ipython-notebooks/basemap-maps/
# Make shortcut to Basemap object, 
# not specifying projection type for this example
m = Basemap() 

# Make trace-generating function (return a Scatter object)
def make_scatter(x,y):
    return Scatter(
        x=x,
        y=y,
        mode='lines',
        line=Line(color="black"),
        name=' '  # no name on hover
    )
# Functions converting coastline/country polygons to lon/lat traces
def polygons_to_traces(poly_paths, N_poly):
    ''' 
    pos arg 1. (poly_paths): paths to polygons
    pos arg 2. (N_poly): number of polygon to convert
    '''
    traces = []  # init. plotting list 

    for i_poly in range(N_poly):
        poly_path = poly_paths[i_poly]
        
        # get the Basemap coordinates of each segment
        coords_cc = np.array(
            [(vertex[0],vertex[1]) 
             for (vertex,code) in poly_path.iter_segments(simplify=False)]
        )
        
        # convert coordinates to lon/lat by 'inverting' the Basemap projection
        lon_cc, lat_cc = m(coords_cc[:,0],coords_cc[:,1], inverse=True)
        
        # add plot.ly plotting options
        traces.append(make_scatter(lon_cc,lat_cc))
     
    return traces

# Function generating coastline lon/lat traces
def get_coastline_traces():
    poly_paths = m.drawcoastlines().get_paths() # coastline polygon paths
    N_poly = 91  # use only the 91st biggest coastlines (i.e. no rivers)
    return polygons_to_traces(poly_paths, N_poly)

# Function generating country lon/lat traces
def get_country_traces():
    poly_paths = m.drawcountries().get_paths() # country polygon paths
    N_poly = len(poly_paths)  # use all countries
    return polygons_to_traces(poly_paths, N_poly)

In [23]:
# Get list of of coastline and country lon/lat traces
traces_cc = get_coastline_traces()+get_country_traces()

In [ ]:
data = Data([trace1] + traces_c)
layout = {
  "autosize": False, 
  "height": 800, 
  "legend": {
    "x": 0.861031518625, 
    "y": 0.1
  }, 
  "title": "Temperature at +00h, GFS 1deg Forecast, UTC Start time : 31/01/2018", 
  "width": 1200, 
  "xaxis": {
    "autorange": False, 
    "gridwidth": 1.2, 
    "mirror": True, 
    "range": [lo1, lo2], 
    "showline": True, 
    "ticks": "", 
    "title": "Longitude", 
    "type": "linear", 
    "zeroline": False
  }, 
  "yaxis": {
    "autorange": False, 
    "gridwidth": 1.2, 
    "mirror": True, 
    "range": [la2, la1], 
    "showline": True, 
    "ticks": "", 
    "title": "Latitude", 
    "type": "linear", 
    "zeroline": False
  }
}
fig = Figure(data=data, layout=layout)
plot_url = plotly.offline.plot(fig)